In [1]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [15]:
robot.getStates()

{'q': array([ 1.8466009 ,  0.90860445, -0.815622  , -1.95736279,  1.0293229 ,
         2.40330383,  1.25407804]),
 'dq': array([ 0.00081562,  0.00058247,  0.00251157, -0.00068401,  0.00012018,
        -0.00037878,  0.0004964 ]),
 'T': array([  0.11434627, -37.77398682, -17.52150536,  13.33224964,
          0.55681074,   2.19449592,  -0.04346249])}

In [3]:
import numpy as np
from FR3Py.robot.model_collision_avoidance import PinocchioModel
pin_robot = PinocchioModel()
joint_lb = np.array([-2.3093, -1.5133, -2.4937, -2.7478, -2.48, 0.8521, -2.6895])
joint_ub = np.array([2.3093, 1.5133, 2.4937, -0.4461, 2.48, 4.2094, 2.6895])
q_nominal = np.array([0.7237753, 0.80070632, 0.14942601, -1.71996101, -0.16544614,
         2.48563189,  1.75224705, 0.025, 0.025])
pin_info = pin_robot.getInfo(q_nominal, np.zeros_like(q_nominal))
P_EE_desired = pin_info['P_EE']
R_EE_desired = np.array([[1, 0, 0],
                        [0, -1, 0],
                        [0, 0, -1]])


In [4]:
q_initial = np.array([ 1.99664562e-04, -7.83964837e-01,  3.22413486e-03, -2.38147407e+00,
        -7.03871748e-04,  1.57095859e+00,  7.86564759e-01, 0.025, 0.025])
pin_info = pin_robot.getInfo(q_initial, np.zeros_like(q_initial))
P_EE_initial = pin_info['P_EE']

In [13]:
import time
from cores.utils.control_utils import get_torque_to_track_traj_const_ori

T = 10
n_joints = 7
q_bar = 0.5*(joint_ub + joint_lb)
# Kp = np.diag([20,20,20,100,100,100])
# Kd = np.diag([20,20,20,100,100,100])

Kp_task = np.diag([2,2,2,10,10,10])*10
Kd_task = np.diag([2,2,2,10,10,10])*10

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos = P_EE_initial + (P_EE_desired - P_EE_initial)*t/T
    traj_vel = (P_EE_desired - P_EE_initial)/T
    traj_acc = np.zeros(3)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    Minv = pin_info["Minv"][0:n_joints,0:n_joints]  # shape (7,7)
    M = pin_info["M"][0:n_joints,0:n_joints]  # shape (7,7)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint
    robot.setCommands(u_nominal)






KeyboardInterrupt: 